# Closed Systems QFlux Demonstration Notebook

In [ ]:
!pip install qflux

In [ ]:
!apt-get install texlive-latex-base texlive-latex-extra cm-super dvipng

In [ ]:
# Imports
import matplotlib.pyplot as plt
import qutip as qt
import numpy as np
# Jupyter Plotting Preamble...
use_latex = True
# To use Latex:
if use_latex:
    plt.rcParams['text.usetex']      = True
    plt.rcParams['font.family']      = 'Computer Modern Serif'
    plt.rcParams['font.serif']       = ['Computer Modern Roman']
    plt.rcParams['mathtext.bf']      = 'serif:bold'
    plt.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams['mathtext.default'] = 'it'
# To not use Latex:
else:
    plt.rcParams['text.usetex']      = False
    plt.rcParams['font.family']      = 'serif'
    plt.rcParams['font.serif']       = 'Palatino'

# Figure Saving Parameters
plt.rcParams['savefig.dpi']         = 300        # DPI of figure output
plt.rcParams['savefig.transparent'] = True       # Save figure with transparent background
#----------------- Figure Size ------------------#
plt.rcParams['figure.figsize']      = (11.7/1.5, 8.3/1.5)# Figure Size - Adjust as necessary
#------------------ Line Plot -------------------#
plt.rcParams['lines.markersize']    = 4          # Marker size for scatterplots
plt.rcParams['lines.linewidth']     = 1.2        # linewidth
#------------------ Font Size -------------------#
plt.rcParams['font.size']           = 20         # default font size
plt.rcParams['legend.fontsize']     = 20         # Font size for legend
plt.rcParams['axes.labelsize']      = 22         # Font size for tick labels (ticklabels)
plt.rcParams['axes.titlesize']      = 22         # Font size for axis labels (.xlabel/.ylabel)
plt.rcParams['image.aspect']        = 1.0        # Image Aspect Ratio (plt.imshow)
plt.rcParams['image.resample']      = False      # Whether to resample
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']   # Display figures inline as SVG (or 'pdf', 'png', ...)

# Quantum Dynamics of a Harmonic Oscillator

<a id='Script3pt1'></a>
## Script 3.1:

In [ ]:
from qflux.closed_systems import DynamicsCS

qho_dyn_obj = DynamicsCS(n_basis=128, xo=1.0, po=0.0, mass=1.0, omega=1.0)
qho_dyn_obj.set_coordinate_operators()
qho_dyn_obj.initialize_operators()
qho_dyn_obj.set_initial_state(wfn_omega=0.2)

total_time = 20.0
N_steps    = 400

qho_dyn_obj.set_propagation_time(total_time, N_steps)
qho_dyn_obj.set_hamiltonian(potential_type='harmonic')
qho_dyn_obj.propagate_qt()
qho_dyn_obj.propagate_SOFT()


In [ ]:
exp_x_qt = qt.expect(qho_dyn_obj.x_op, qho_dyn_obj.dynamics_results_op)
exp_p_qt = qt.expect(qho_dyn_obj.p_op, qho_dyn_obj.dynamics_results_op)

exp_x_ana = [ qho_dyn_obj.xo*np.cos(qho_dyn_obj.omega*t) + (qho_dyn_obj.po/qho_dyn_obj.mass/qho_dyn_obj.omega)*np.sin(qho_dyn_obj.omega*t) for t in qho_dyn_obj.tlist]
exp_p_ana = [ qho_dyn_obj.po*np.cos(qho_dyn_obj.omega*t) -qho_dyn_obj.xo*qho_dyn_obj.omega*qho_dyn_obj.mass*np.sin(qho_dyn_obj.omega*t)  for t in qho_dyn_obj.tlist]

# Plot expectation value, compare to analytic expression
plt.figure(figsize=(9, 6.5))
plt.plot(qho_dyn_obj.tlist, exp_x_qt, label=r'$\left\langle x \right\rangle$ (QuTiP)', color='dodgerblue')
plt.plot(qho_dyn_obj.tlist, exp_x_ana, label=r'$\left\langle x \right\rangle$ (Analytic)',
         lw=0, marker='x', markevery=10, color='dodgerblue', ms=8)
plt.plot(qho_dyn_obj.tlist, exp_p_qt, label=r'$\left\langle p \right\rangle$ (QuTiP)', color='crimson')
plt.plot(qho_dyn_obj.tlist, exp_p_ana, label=r'$\left\langle p \right\rangle$ (Analytic)',
         lw=0, marker='x', markevery=10, color='crimson', ms=8)
plt.ylim(-1.55, 1.70)

plt.legend(ncols=2, loc='upper center')
plt.hlines([-1, 0, 1], min(qho_dyn_obj.tlist), max(qho_dyn_obj.tlist), ls='--', lw=0.85, color='tab:grey', zorder=2)
plt.xlim(min(qho_dyn_obj.tlist), max(qho_dyn_obj.tlist))
plt.xlabel('Time')

<a id='Script3pt2'></a>
## Script 3.2:

In [ ]:
import qutip as qt
import numpy as np

# Define the system parameters
mass = 1.0
hbar = 1.0
omega = 1.0

# Initial state: coherent state with amplitude $\alpha = (x0 + i p_0)/\sqrt{2}$
x_0, p_0 = 1.0, 0.0
N = 128  # Number of basis states
psi_0 = qt.coherent(N, alpha=(x_0 + 1.j*p_0)/np.sqrt(2))

# Time grid
n_steps, total_time = 400, 20.0
tlist = np.linspace(0, total_time, n_steps)

# Define the Hamiltonian
a = qt.destroy(N)
H_ho = hbar * omega * (a.dag() * a + 0.5)

# Propagate using the Runge–Kutta solver
solver_options = {'nsteps': len(tlist), 'progress_bar': True}
result = qt.sesolve(H_ho, psi_0, tlist, options=solver_options)


<a id='Script3pt3'></a>
## Script 3.3:

In [ ]:

import matplotlib.pyplot as plt

# Operators for position and momentum
X_op = (a.dag() + a) / np.sqrt(2)
P_op = 1j * (a.dag() - a) / np.sqrt(2)

# Compute numerical expectation values
exp_x_qt = qt.expect(X_op, result.states)
exp_p_qt = qt.expect(P_op, result.states)

# Analytical results
exp_x_ana = [x_0*np.cos(omega*t) + (p_0/mass/omega)*np.sin(omega*t) for t in tlist]
exp_p_ana = [-mass*omega*x_0*np.sin(omega*t) + p_0*np.cos(omega*t) for t in tlist]

# Plot
fig, ax = plt.subplots()
ax.plot(tlist, exp_x_ana, '-', color='blue', label=r'$\left\langle x \right\rangle$ (Analytical)')
ax.plot(tlist, exp_x_qt, 'o', color='blue', label=r'$\left\langle x \right\rangle$ (QuTiP)',
        markeredgecolor='blue', markevery=4, fillstyle='full', markerfacecolor='white')
ax.plot(tlist, exp_p_ana, '-', color='red', label=r'$\left\langle p \right\rangle$ (Analytical)')
ax.plot(tlist, exp_p_qt, 'o', color='red', label=r'$\left\langle p \right\rangle$ (QuTiP)',
        markeredgecolor='red', markevery=4, fillstyle='full', markerfacecolor='white')
ax.axhline(0, ls='--', lw=0.5, color='black', alpha=0.5)
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('Expectation Value')
plt.legend(loc='upper center', ncol=2)
ax.set_ylim(-1.5, 1.825)
plt.hlines([-1, 0, 1], min(tlist), max(tlist), ls='--', lw=0.85, color='tab:grey', zorder=2)
ax.set_xlim(min(tlist), max(tlist))
plt.show()



<a id='Script3pt4'></a>
## Script 3.4:

In [ ]:

import numpy as np

def get_xgrid(xmin, xmax, N_pts):
    """Generate an evenly spaced position grid."""
    dx = (xmax - xmin)/N_pts
    xgrid = np.arange(-N_pts/2, N_pts/2)*dx
    return xgrid

def get_pgrid(xmin, xmax, N_pts, reorder=True):
    """Generate a momentum grid using FFT-compatible ordering."""
    dp = 2 * np.pi / (xmax-xmin)
    pmin = -dp * N_pts / 2
    pmax = dp * N_pts / 2
    plus_pgrid = np.linspace(0, pmax, N_pts//2+1)
    minus_pgrid = - np.flip(np.copy(plus_pgrid))
    if reorder:
        pgrid = np.concatenate((plus_pgrid[:-1], minus_pgrid[:-1]))
    else:
        pgrid = np.concatenate((minus_pgrid, plus_pgrid))
    return pgrid

def get_coherent_state(x, p_0, x_0, mass=1, omega=1, hbar=1):
    """Generate an initial coherent state wavefunction."""
    normalization = (mass*omega/np.pi/hbar)**(0.25)
    y = normalization*np.exp(-1*(mass*omega/hbar/2)*((x-x_0)**2)+1j*p_0*x/hbar)
    return y

xmin = -7.0
xmax = 7.0
N_pts = 128
mass = 1.0  # mass in atomic units
omega = 1.0  # oscillator frequency
xgrid = get_xgrid(xmin, xmax, N_pts)
dx = xgrid[1] - xgrid[0]
pgrid = get_pgrid(xmin, xmax, N_pts, reorder=True)

x_0 = 1.0
p_0 = 0.0
psi_0 = get_coherent_state(xgrid, p_0, x_0, mass, omega)



<a id='Script3pt5'></a>
## Script 3.5:

In [ ]:
import numpy as np

def get_harmonic_potential(x, x_0=0.0, mass=1, omega=1):
    return mass * omega**2 * (x - x_0)**2 / 2

def get_kinetic_energy(p, mass=1):
    return p**2 / (2 * mass)

Vx_harm = get_harmonic_potential(xgrid)
K_harm = get_kinetic_energy(pgrid, mass)


<a id='Script3pt6'></a>
## Script 3.6:

In [ ]:
import numpy as np
from tqdm.auto import trange

def get_propagator_on_grid(operator_grid, tau, hbar=1):
    return np.exp(-1.0j * operator_grid * tau / hbar)

def do_SOFT_propagation(psi, K_prop, V_prop):
    psi_t_position_grid = V_prop * psi
    psi_t_momentum_grid = K_prop * np.fft.fft(psi_t_position_grid, norm="ortho")
    psi_t = V_prop * np.fft.ifft(psi_t_momentum_grid, norm="ortho")
    return psi_t

tmin, tmax, N_tsteps = 0.0, 20.0, 400
tgrid = np.linspace(tmin, tmax, N_tsteps)
tau = tgrid[1] - tgrid[0]

V_prop = get_propagator_on_grid(Vx_harm/2, tau)
K_prop = get_propagator_on_grid(K_harm, tau)

propagated_states_harm = [psi_0]
psi_t = psi_0
for _ in trange(len(tgrid)):
    psi_t = do_SOFT_propagation(psi_t, K_prop, V_prop)
    propagated_states_harm.append(psi_t)

propagated_states_harm = np.asarray(propagated_states_harm)[:-1]


<a id='Script3pt7'></a>
## Script 3.7:

In [ ]:
def position_expectation_value(xgrid, psi):
    dx = xgrid[1]-xgrid[0]
    return dx*np.real(np.sum(xgrid * np.conjugate(psi) * psi))

def momentum_expectation_value(dx, pgrid, psi):
    psip = np.fft.fft(psi)
    return dx*np.real(np.sum(pgrid * np.conjugate(psip) * psip))/len(psi)

avx_soft = [position_expectation_value(xgrid, propagated_states_harm[i]) for i in range(len(propagated_states_harm))]
dx = xgrid[1]-xgrid[0]
avp_soft = [momentum_expectation_value(dx, pgrid, propagated_states_harm[i]) for i in range(len(propagated_states_harm))]

avx_ana = [x_0*np.cos(omega*t) + (p_0/mass/omega)*np.sin(omega*t) for t in tgrid]
avp_ana = [-x_0*omega*mass*np.sin(omega*t) + p_0*np.cos(omega*t) for t in tgrid]

# Plot
fig, ax = plt.subplots()
ax.plot(tlist, avx_ana, '-', color='blue', label=r'$\left\langle x \right\rangle$ (Analytical)')
ax.plot(tlist, avx_soft, 'o', color='blue', label=r'$\left\langle x \right\rangle$ (SOFT)',
        markeredgecolor='blue', markevery=4, fillstyle='full', markerfacecolor='white')
ax.plot(tlist, avp_ana, '-', color='red', label=r'$\left\langle p \right\rangle$ (Analytical)')
ax.plot(tlist, avp_soft, 'o', color='red', label=r'$\left\langle p \right\rangle$ (SOFT)',
        markeredgecolor='red', markevery=4, fillstyle='full', markerfacecolor='white')
ax.axhline(0, ls='--', lw=0.5, color='black', alpha=0.5)
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('Expectation Value')
plt.legend(loc='upper center', ncol=2)
ax.set_ylim(-1.5, 1.825)
plt.hlines([-1, 0, 1], min(tlist), max(tlist), ls='--', lw=0.85, color='tab:grey', zorder=2)
ax.set_xlim(min(tlist), max(tlist))
plt.show()

<a id='Script4pt1'></a>
## Script 4.1:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import axes
from tqdm.notebook import trange
import scipy.linalg as LA

from qiskit.circuit.library import QFT
from qiskit_aer import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit_ibm_runtime import QiskitRuntimeService, Options, SamplerV2


<a id='Script4pt2'></a>
## Script 4.2:

In [ ]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from IPython.display import display

# Create a quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

# Apply Hadamard gate to the first qubit
qc.h(0)

# Apply CNOT gate with qubit 0 as control and qubit 1 as target
qc.cx(0, 1)

# Measure the qubits
qc.measure([0, 1], [0, 1])

# Visualize the circuit
display(qc.draw('mpl'))

# Simulate the circuit
simulator = Aer.get_backend('aer_simulator')
compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit).result()

# Get the measurement results
counts = result.get_counts()
fig, ax=plt.subplots()
plot_histogram(counts, ax=ax)

<a id='Script4pt3'></a>
## Script 4.3:

In [ ]:
from qflux.closed_systems.utils import convert_fs_to_au, convert_eV_to_au, convert_au_to_fs
# Conversion factor
ev2au = convert_eV_to_au(1.0)

def get_doublewell_potential(x, x0=1.9592, f=ev2au, a0=0.0, a1=0.429, a2=-1.126, a3=-0.143, a4=0.563):
    # A-T pair double-well potential in Hartrees (x is in Bohr)
    xi = x/x0
    return f*(a0 + a1*xi + a2*xi**2 + a3*xi**3 + a4*xi**4)

def get_doublewell_potential_second_deriv(x, x0=1.9592, f=ev2au, a0=0.0, a1=0.429, a2=-1.126, a3=-0.143, a4=0.563):
    # A-T pair double-well potential in Hartrees (x is in Bohr)
    return f*(2*a2/x0**2 + 6*a3*x/x0**3 + 12*a4*x**2/x0**4)


<a id='Script4pt4'></a>
## Script 4.4:

In [ ]:
from qflux.closed_systems import QubitDynamicsCS
from qflux.closed_systems.utils import get_proton_mass
from qiskit_aer import Aer

proton_mass = get_proton_mass()
x0      = 1.9592
N_steps = 3000

# frequency corresponding to the right well (in atomic units)
omega = np.sqrt(get_doublewell_potential_second_deriv(x0)/proton_mass)
AT_dyn_obj  = QubitDynamicsCS(n_basis=64, xo=1.5*x0, mass=proton_mass, omega=omega)

AT_dyn_obj.set_coordinate_operators(x_min=-4.0, x_max=4.0)
AT_dyn_obj.initialize_operators()
AT_dyn_obj.set_initial_state(wfn_omega=omega)

total_time = 30.0 * convert_fs_to_au(1.0)
AT_dyn_obj.set_propagation_time(total_time, N_steps)
AT_dyn_obj.set_hamiltonian(potential_type='quartic')

AT_dyn_obj.propagate_SOFT()
AT_dyn_obj.propagate_qt()

# Instantiate a backend object:
backend = Aer.get_backend('statevector_simulator')
AT_dyn_obj.propagate_qSOFT(backend=backend) # always shows progress bar


<a id='Script4pt5'></a>
## Script 4.5:

In [ ]:
# initial coherent state in the right well centered at x/x0 = 1.5
mass_proton = 1836.15 # proton mass in units of electron mass
x0 = 1.9592
x_0 = 1.5*x0
p_0 = 0.0
# Define xgrid:
xmin = -4.
xmax = 4.
Nq = 6 # number of qubits
N_xpts = 2**Nq
xgrid = get_xgrid(xmin, xmax, N_xpts)
omega = np.sqrt(get_doublewell_potential_second_deriv(x0)/proton_mass)

psi_0 = get_coherent_state(xgrid, p_0, x_0, proton_mass, omega)


<a id='Script4pt6'></a>
## Script 4.6:

In [ ]:
from qflux.closed_systems.utils import convert_fs_to_au, convert_au_to_fs

au2fs = convert_au_to_fs(1.0)
fs2au = convert_fs_to_au(1.0)

# grid preparation and spacing
pgrid = get_pgrid(xmin, xmax, N_xpts, reorder=True)

dx = xgrid[1] - xgrid[0]
dp = pgrid[1] - pgrid[0]

Vx_DW = get_doublewell_potential(xgrid)
VV = Vx_DW - get_doublewell_potential(x0) - omega/2

tmin = 0.0
tmax = 30.0*fs2au
iterations = 3000
mass = mass_proton

# time grid preparation and spacing
tgrid = np.linspace(tmin, tmax, iterations)
time_step = tgrid[1] - tgrid[0]

# propagators (note that potential energy is divided by two)
VVd_prop = np.diag(np.exp(-1j*Vx_DW/2*time_step))
KEd_prop = np.diag(np.exp(-1j*pgrid**2/2/mass*time_step))


<a id='Script4pt7'></a>
## Script 4.7:

In [ ]:
import scipy.linalg as LA
from qiskit.circuit.library import QFT
from qiskit_aer import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit_ibm_runtime import QiskitRuntimeService, Options, SamplerV2
from tqdm.notebook import trange

# Initialize an Empty Circuit
nqubits = Nq
q_reg = QuantumRegister(nqubits)
c_reg = ClassicalRegister(nqubits)
qc = QuantumCircuit(q_reg)

qc.initialize(psi_0, q_reg[:],normalize=True)

for k in trange(iterations):
    V_op = Operator(VVd_prop)
    qc.append(V_op, q_reg)
    qc.append(QFT(nqubits,do_swaps=True,inverse=False),q_reg)
    K_op = Operator(KEd_prop)
    qc.append(K_op, q_reg)
    qc.append(QFT(nqubits,do_swaps=True,inverse=True),q_reg)
    qc.append(V_op, q_reg)


<a id='Script4pt8'></a>
## Script 4.8:

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qflux.closed_systems.custom_execute import execute


backend = Aer.get_backend('statevector_simulator')
executed_circuit = execute(qc, backend=backend, shots=1024)

psin = executed_circuit.result().get_statevector().data


<a id='Script4pt9'></a>
## Script 4.9:

In [ ]:
# SOFT propagation
# propagators (note that potential energy is divided by two)

V_prop = np.exp(-1j*Vx_DW/2*time_step)
K_prop = np.exp(-1j*pgrid**2/2/mass*time_step)


propagated_states = [psi_0]
psi_t = psi_0
print("For ",tmax*au2fs," fs using a timestep of ",time_step*au2fs," fs = ",time_step," a.u.")

for tstep_idx in trange(len(tgrid)):
    psi_t = do_SOFT_propagation(psi_t, K_prop, V_prop)
    propagated_states.append(psi_t)

propagated_states = np.asarray(propagated_states)[:-1]


<a id='Script4pt10'></a>
## Script 4.10:

In [ ]:
# Visualization
from scipy.interpolate import interp1d
def get_prob_density(psi):
    return np.real(np.conjugate(psi) * psi)


x_dense = np.linspace(xgrid[0], xgrid[-1], 512)
f_interp = interp1d(xgrid, get_prob_density(propagated_states[-1]), kind='cubic')
rho_interp = f_interp(x_dense)

fig, ax = plt.subplots()
ax.plot(xgrid, VV, '-',color='black',label='A-T pair potential')
ax.plot(xgrid, 0.04*np.real(get_prob_density(psi_0)),'--',color='red',label='Initial coherent state')
ax.plot(x_dense, 0.04*rho_interp,'-',color='blue',label='(SOFT) State at t = 30 fs',zorder=0,markeredgecolor='blue',fillstyle='full',markerfacecolor='white')
ax.plot(xgrid, 0.04*np.real(psin.conj()*psin/dx),'o',color='blue',label='(Qiskit) State at t = 30 fs', markevery=1, alpha=0.25)
ax.axhline(0, lw=0.5, color='black', alpha=1.0)
ax.axvline(-x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(x0, lw=0.5, color='black', alpha=0.5)
ax.axvline(x0*1.5, lw=0.5, color='red', alpha=0.5)
ax.set_xlabel('x, Bohr',fontsize=14)
ax.set_ylabel('Energy, Hartrees',fontsize=14)
ax.tick_params(labelsize=12, grid_alpha=0.5)
plt.ylim(-0.03,0.07)
plt.legend(fontsize=12,loc='upper center')
plt.show()


<a id='Script4pt11'></a>
## Script 4.11:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

def exp_all_z(circuit, quantum_register, pauli_indexes, control_qubit=None, t=1):
    r"""
    Implements \( e^{-i t Z \otimes \cdots \otimes Z} \) on specified qubits.

    Args:
        circuit (QuantumCircuit): The circuit to modify.
        quantum_register (QuantumRegister): Register containing target qubits.
        pauli_indexes (list): Indices of qubits where \( Z \) acts.
        control_qubit (Qubit, optional): Optional control qubit for conditional application.
        t (float): Evolution time.

    Returns:
        QuantumCircuit: Updated circuit with the operation applied.
    """
    if control_qubit and control_qubit.register not in circuit.qregs:
        circuit.add_register(control_qubit.register)

    if not pauli_indexes:
        if control_qubit:
            circuit.p(t, control_qubit)  # Phase gate
        return circuit

    # Parity computation
    for i in range(len(pauli_indexes) - 1):
        circuit.cx(quantum_register[pauli_indexes[i]], quantum_register[pauli_indexes[i + 1]])

    # Apply phase rotation
    target = quantum_register[pauli_indexes[-1]]
    angle = -2 * t
    if control_qubit:
        circuit.crz(angle, control_qubit, target)
    else:
        circuit.rz(angle, target)

    # Uncompute parity
    for i in reversed(range(len(pauli_indexes) - 1)):
        circuit.cx(quantum_register[pauli_indexes[i]], quantum_register[pauli_indexes[i + 1]])

    return circuit


<a id='Script4pt12'></a>
## Script 4.12:

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister

def exp_pauli(pauli, quantum_register, control_qubit=None, t=1):
    r"""
    Implements \( e^{-i t P} \) for a Pauli string \( P \).

    Args:
        pauli (str): Pauli string (e.g., "XIZY").
        quantum_register (QuantumRegister): Target register.
        control_qubit (Qubit, optional): Optional control qubit.
        t (float): Evolution time.

    Returns:
        QuantumCircuit: Circuit implementing the Pauli evolution.
    """
    if len(pauli) != len(quantum_register):
        raise ValueError("Pauli string length must match register size.")

    pauli_indexes = []
    pre_circuit = QuantumCircuit(quantum_register)

    for i, op in enumerate(pauli):
        if op == 'I':
            continue
        elif op == 'X':
            pre_circuit.h(i)
            pauli_indexes.append(i)
        elif op == 'Y':
            pre_circuit.rx(np.pi/2, i)
            pauli_indexes.append(i)
        elif op == 'Z':
            pauli_indexes.append(i)
        else:
            raise ValueError(f"Invalid Pauli operator '{op}' at position {i}.")

    circuit = QuantumCircuit(quantum_register)
    circuit.compose(pre_circuit, inplace=True)
    circuit = exp_all_z(circuit, quantum_register, pauli_indexes, control_qubit, t)
    circuit.compose(pre_circuit.inverse(), inplace=True)
    return circuit


<a id='Script4pt13'></a>
## Script 4.13:

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

def hamiltonian_simulation(hamiltonian, quantum_register=None, control_qubit=None, t=1, trotter_number=1):
    r"""
    Implements \( e^{-i H t} \) using first-order Trotterization.

    Args:
        hamiltonian (dict): Pauli terms with coefficients (e.g., {"ZZ": 0.5, "XX": 0.3}).
        quantum_register (QuantumRegister, optional): Target register.
        control_qubit (Qubit, optional): Optional control qubit.
        t (float): Simulation time.
        trotter_number (int): Number of Trotter steps.

    Returns:
        QuantumCircuit: Trotterized Hamiltonian evolution circuit.
    """
    if not hamiltonian:
        raise ValueError("Hamiltonian must contain at least one term.")

    n_qubits = len(next(iter(hamiltonian)))
    if quantum_register is None:
        quantum_register = QuantumRegister(n_qubits)

    delta_t = t / trotter_number
    circuit = QuantumCircuit(quantum_register)

    for pauli_str, coeff in hamiltonian.items():
        term_circuit = exp_pauli(pauli_str, quantum_register, control_qubit, coeff * delta_t)
        circuit.compose(term_circuit, inplace=True)

    full_circuit = QuantumCircuit(quantum_register)
    for _ in range(trotter_number):
        full_circuit.compose(circuit, inplace=True)

    return full_circuit


<a id='Script4pt14'></a>
## Script 4.14:

In [ ]:
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram

# Initialize circuit
qr=QuantumRegister(2)
qc = QuantumCircuit(qr)

# Define Hamiltonian: H = 0.5 * ZZ + 0.3 * YY
hamiltonian = {
    "ZZ": 0.5,
    "YY": 0.3,
}
# Build the Hamiltonian evolution circuit
t = np.pi / 4  # evolution time
trotter_steps = 1
U = hamiltonian_simulation(hamiltonian, quantum_register=qr, t=t, trotter_number=trotter_steps)

# Initialize a uniform superposition, add propagator and measurements
qc.h(qr)
qc.append(U,qr)
qc.measure_all()

# Simulate circuit
sim = Aer.get_backend("aer_simulator")
qobj = transpile(qc, sim)
result = sim.run(qobj).result()
counts = result.get_counts()
# Plot the result
print("Measurement counts:", counts)

# Draw circuit
qc.decompose().draw()


<a id='Script4pt15'></a>
## Script 4.15:

In [ ]:
J = 1
h0 = -0.5
h1 = 0.5
X = np.array([[0,1],[1,0]], dtype = complex)
Y = np.array([[0,1j],[-1j,0]], dtype = complex)
Z = np.array([[1,0],[0,-1]], dtype = complex)
I = np.eye(2, dtype = complex)
H = 0.5*(h0*np.kron(Z, I) + h1*np.kron(I, Z)) + J/4*(np.kron(X, X) + np.kron(Y, Y) + np.kron(Z, Z))
U = LA.expm(-1j * H)


<a id='Script4pt16'></a>
## Script 4.16:

In [ ]:
psi_init = np.array([1,0,0,0],dtype = complex)
psi_fin = U @ psi_init


<a id='Script4pt17'></a>
## Script 4.17:

In [ ]:
qreg = QuantumRegister(2)
creg = ClassicalRegister(2, 'creg')
entangler = QuantumCircuit(qreg, creg)
# Qiskit initializes qubits in |00> by default


<a id='Script4pt18'></a>
## Script 4.18:

In [ ]:
U_gate = Operator(U)
entangler.append(U_gate, [0,1])


<a id='Script4pt19'></a>
## Script 4.19:

In [ ]:
entangler.measure(0,0)
entangler.measure(1,1)


<a id='Script4pt20'></a>
## Script 4.20:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

MY_API_TOKEN = "INSERT_YOUR_API_TOKEN_HERE"
MY_INSTANCE = "ibm-q/open/main"

service = QiskitRuntimeService(channel="ibm_cloud",
                               token=MY_API_TOKEN,
                               instance=MY_INSTANCE)

from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def run_IBM_session(circuit, backend, nshots=2048, opt_level=1):
    # Transpilation to device gate-set/architecture
    pm = generate_preset_pass_manager(backend=backend,
                                      optimization_level=opt_level)
    transpiled_circuit = pm.run(circuit)

    # Circuit execution
    with Session(backend=backend) as session:
        sampler = Sampler(mode=session)
        sampler.options.default_shots = nshots
        job = sampler.run([transpiled_circuit])

    # Result retrieval
    print(f"Job ID is {job.job_id()}")
    pub_result = job.result()[0]
    result_dict = pub_result.data.creg.get_counts()

    return result_dict


<a id='Script4pt21'></a>
## Script 4.21:

In [ ]:
from qiskit_aer import AerSimulator
result_dict = run_IBM_session(entangler, backend=AerSimulator())
print("Counts per state:", result_dict)


<a id='Script4pt22'></a>
## Script 4.22:

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
noisy_backend = FakeManilaV2()
noisy_result_dict = run_IBM_session(entangler, backend=noisy_backend)
print("Counts per state (noisy backend):", noisy_result_dict)


<a id='Script4pt23'></a>
## Script 4.23:

In [ ]:
from qflux.closed_systems.spin_dynamics_oo import SpinDynamicsS

num_q = 3
evolution_timestep = 0.1
n_trotter_steps = 1
hamiltonian_coefficients = [[0.75 / 2, 0.75 / 2, 0.0, 0.65]] + [
    [0.5, 0.5, 0.0, 1.0] for _ in range(num_q - 1)
]
initial_state = "011"  # Specify the initial state as a binary string

csimulation = SpinDynamicsS(
    num_q,
    evolution_timestep,
    n_trotter_steps,
    hamiltonian_coefficients,
)
csimulation.run_dynamics(nsteps=250, state_string=initial_state)
csimulation.save_results(f"{num_q}_spin_chain")
csimulation.plot_results(f"{num_q}_spin_chain_statevector")


<a id='Script4pt24'></a>
## Script 4.24:

In [ ]:
from qiskit.quantum_info import SparsePauliOp

def get_hamiltonian_n_site_terms(n, coeff, n_qubits):
    '''
        Assemble each term in the Hamiltonian using its Pauli-string
        representation and multiply by the corresponding coefficient.
        coeff = [Jxx, Jyy, Jzz, Oz]
    '''
    XX_coeff, YY_coeff, ZZ_coeff, Z_coeff = coeff

    XX_term = SparsePauliOp("I"*n + "XX" + "I"*(n_qubits - 2 - n)) * XX_coeff
    YY_term = SparsePauliOp("I"*n + "YY" + "I"*(n_qubits - 2 - n)) * YY_coeff
    ZZ_term = SparsePauliOp("I"*n + "ZZ" + "I"*(n_qubits - 2 - n)) * ZZ_coeff
    Z_term  = SparsePauliOp("I"*n + "Z"  + "I"*(n_qubits - 1 - n)) * Z_coeff

    return XX_term + YY_term + ZZ_term + Z_term


<a id='Script4pt25'></a>
## Script 4.25:

In [ ]:
def get_heisenberg_hamiltonian(n_qubits, coeff=None):
    r'''
    Constructs the Heisenberg Hamiltonian for an N-site spin chain.

    H = \sum _i ^N h_z Z_i
        + \sum _i ^{N-1} (h_xx X_iX_{i+1}
            + h_yy Y_iY_{i+1}
            + h_zz Z_iZ_{i+1}
            )

    Parameters:
        n_qubits (int): Number of spins/qubits.
        coeff (list of lists, optional): A list of sublists containing the coefficients
            [XX, YY, ZZ, Z] for each site. The last sublist contains only the Z component.
            Defaults to uniform coefficients if not provided.

    Returns:
        list: Two components of the Hamiltonian (even and odd terms).
    '''

    # Three qubits because for 2 we get H_O = 0
    assert n_qubits >= 3

    if coeff == None:
        'Setting default values for the coefficients'
        coeff = [[1.0, 1.0, 1.0, 1.0] for i in range(n_qubits)]

    # Even terms of the Hamiltonian
    # (summing over individual pair-wise elements)
    H_E = sum((get_hamiltonian_n_site_terms(i, coeff[i], n_qubits)
               for i in range(0, n_qubits-1, 2)))

    # Odd terms of the Hamiltonian
    # (summing over individual pair-wise elements)
    H_O = sum((get_hamiltonian_n_site_terms(i, coeff[i], n_qubits)
               for i in range(1, n_qubits-1, 2)))

    # adding final Z term at the Nth site
    final_term = SparsePauliOp("I" * (n_qubits - 1) + "Z")
    final_term *= coeff[n_qubits-1][3]
    if (n_qubits % 2) == 0:
        H_E += final_term
    else:
        H_O += final_term

    # Returns the list of the two sets of terms
    return [H_E, H_O]


<a id='Script4pt26'></a>
## Script 4.26:

In [ ]:
num_q = 3
# XX YY ZZ, Z
ham_coeffs = ([[0.75/2, 0.75/2, 0.0, 0.65]]+
              [[0.5, 0.5, 0.0, 1.0] for _ in range(num_q-1)])

spin_chain_hamiltonian = get_heisenberg_hamiltonian(num_q, ham_coeffs)

print('Hamiltonian (even and odd components):',spin_chain_hamiltonian)
print('Combined Hamiltonian:', sum(spin_chain_hamiltonian))


<a id='Script4pt27'></a>
## Script 4.27:

In [ ]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
from itertools import groupby
import re

def get_time_evolution_operator(num_qubits, tau, trotter_steps, coeff=None):
    '''
    Generates the Trotterized time-evolution operator for a Heisenberg spin chain

        Inputs:
            num_qubits (int): number of qubits, which should be equal to the
                number of spins in the chain
            evo_time (float): time parameter in time-evolution operator
            trotter_steps (int): number of time steps for the Suzuki-Trotter
                decomposition
            coeff (list of lists): parameters for each term in the Hamiltonian
                for each site ie ([[XX0, YY0, ZZ0, Z0], [XX1, YY1, ZZ1, Z1], ...])
        Returns:
            evo_op.definition: Trotterized time-evolution operator
    '''
    # Heisenberg_hamiltonian = [H_E, H_O]
    heisenberg_hamiltonian = get_heisenberg_hamiltonian(num_qubits, coeff)

    # e^ (-i*H*evo_time), with Trotter decomposition
    # exp[(i*evo_time)*(IIIIXXIIII + IIIIYYIIII + IIIIZZIIII + IIIIZIIIII)]
    evo_op = PauliEvolutionGate(heisenberg_hamiltonian, tau,
                                synthesis=SuzukiTrotter(order=2,
                                reps=trotter_steps))
    return evo_op.definition

num_shots = 100
num_q = 3
evolution_timestep = 0.1
n_trotter_steps = 1
# XX YY ZZ, Z
ham_coeffs = ([[0.75/2, 0.75/2, 0.0, 0.65]]
                + [[0.5, 0.5, 0.0, 1.0]
                for i in range(num_q-1)])
time_evo_op = get_time_evolution_operator(
    num_qubits=num_q, tau=evolution_timestep,
    trotter_steps=n_trotter_steps, coeff=ham_coeffs)


<a id='Script4pt28'></a>
## Script 4.28:

In [ ]:
def find_string_pattern(pattern, string):
    match_list = []
    for m in re.finditer(pattern, string):
        match_list.append(m.start())
    return match_list

def sort_Pauli_by_symmetry(ham):
    # Separates a qiskit PauliOp object terms into 1 and 2-qubit
    # operators. Furthermore, 2-qubit operators are separated according
    # to the parity of the index first non-identity operation.
    one_qubit_terms = []
    two_qubit_terms = []
    # separating the one-qubit from two-qubit terms
    for term in ham:
        matches = find_string_pattern('X|Y|Z', str(term.paulis[0]))
        pauli_string = term.paulis[0]
        coeff = np.real(term.coeffs[0])
        str_tag = pauli_string.to_label().replace('I', '')
        if len(matches) == 2:
            two_qubit_terms.append((pauli_string, coeff, matches, str_tag))
        elif len(matches) == 1:
            one_qubit_terms.append((pauli_string, coeff, matches, str_tag))

    # sorting the two-qubit terms according to index on which they act
    two_qubit_terms = sorted(two_qubit_terms, key=lambda x: x[2])
    # separating the even from the odd two-qubit terms
    even_two_qubit_terms = list(filter(lambda x: not x[2][0]%2, two_qubit_terms))
    odd_two_qubit_terms = list(filter(lambda x: x[2][0]%2, two_qubit_terms))

    even_two_qubit_terms = [list(v) for i, v in groupby(even_two_qubit_terms, lambda x: x[2][0])]
    odd_two_qubit_terms = [list(v) for i, v in groupby(odd_two_qubit_terms, lambda x: x[2][0])]

    return one_qubit_terms, even_two_qubit_terms, odd_two_qubit_terms


<a id='Script4pt29'></a>
## Script 4.29:

In [ ]:
def generate_circ_pattern_1qubit(circ, term, delta_t):
    coeff = 2 * term[1] * delta_t
    if term[3] == 'X':
        circ.rx(coeff, term[2])
    elif term[3] == 'Y':
        circ.ry(coeff, term[2])
    elif term[3] == 'Z':
        circ.rz(coeff, term[2])
    return circ


<a id='Script4pt30'></a>
## Script 4.30:

In [ ]:
def generate_circ_pattern_2qubit(circ, term, delta_t):

    # wires to which to apply the operation
    wires = term[0][2]

    # angles to parameterize the circuit,
    # based on exponential argument
    if any('XX' in sublist for sublist in term):
        g_phi = ( 2 * (-1) * term[0][1] * delta_t - np.pi / 2)
    else:
        g_phi = - np.pi / 2
    if any('YY' in sublist for sublist in term):
        g_lambda = (np.pi/2 - 2 * (-1) * term[1][1] * delta_t)
    else:
        g_lambda = np.pi/2
    if any('ZZ' in sublist for sublist in term):
        g_theta = (np.pi/2 - 2 * (-1) * term[2][1] * delta_t)
    else:
        g_theta = np.pi/2

    # circuit
    circ.rz(-np.pi/2, wires[1])
    circ.cx(wires[1], wires[0])
    circ.rz(g_theta, wires[0])
    circ.ry(g_phi, wires[1])
    circ.cx(wires[0], wires[1])
    circ.ry(g_lambda, wires[1])
    circ.cx(wires[1], wires[0])
    circ.rz(np.pi/2, wires[0])
    return circ


<a id='Script4pt31'></a>
## Script 4.31:

In [ ]:
def get_manual_Trotter(num_q, pauli_ops, timestep, n_trotter=1,
                       trotter_type='basic', reverse_bits=True):
    # sorts the Pauli strings according to qubit number they affect and symmetry
    one_q, even_two_q, odd_two_q = sort_Pauli_by_symmetry(pauli_ops)
    # scales the timestep according to the number of trotter steps
    timestep_even_two_q = timestep / n_trotter
    timestep_odd_two_q = timestep / n_trotter
    timestep_one_q = timestep / n_trotter
    # symmetric places 1/2 of one_q and odd_two_q before and after even_two_q
    if trotter_type == 'symmetric':
        timestep_odd_two_q /= 2
        timestep_one_q /= 2
    # constructs circuits for each segment of the operators
    qc_odd_two_q, qc_even_two_q, qc_one_q = QuantumCircuit(num_q), QuantumCircuit(num_q), QuantumCircuit(num_q)
    for i in even_two_q:
        qc_even_two_q = generate_circ_pattern_2qubit(qc_even_two_q, i, timestep_even_two_q)
    for i in odd_two_q:
        qc_odd_two_q = generate_circ_pattern_2qubit(qc_odd_two_q, i, timestep_odd_two_q)
    for i in one_q:
        qc_one_q = generate_circ_pattern_1qubit(qc_one_q, i, timestep_one_q)
    # assembles the circuit for Trotter decomposition of exponential
    qr = QuantumRegister(num_q)
    qc = QuantumCircuit(qr)
    if trotter_type == 'basic':
        qc = qc.compose(qc_even_two_q)
        qc = qc.compose(qc_odd_two_q)
        qc = qc.compose(qc_one_q)
    elif trotter_type == 'symmetric':
        qc = qc.compose(qc_one_q)
        qc = qc.compose(qc_odd_two_q)
        qc = qc.compose(qc_even_two_q)
        qc = qc.compose(qc_odd_two_q)
        qc = qc.compose(qc_one_q)
    # repeats the single_trotter circuit several times to match n_trotter
    for i in range(n_trotter-1):
        qc = qc.compose(qc)
    if reverse_bits:
        return qc.reverse_bits()
    else:
        return qc


<a id='Script4pt32'></a>
## Script 4.32:

In [ ]:
spin_chain_hamiltonian = get_heisenberg_hamiltonian(num_q, ham_coeffs)

spin_chain_hamiltonian = sum(spin_chain_hamiltonian)
print(get_manual_Trotter(num_q, spin_chain_hamiltonian, 0.1).draw())
print(get_manual_Trotter(num_q, spin_chain_hamiltonian, 0.1, n_trotter=2).draw())
print(get_manual_Trotter(num_q, spin_chain_hamiltonian, 0.1, trotter_type='symmetric').draw())
print(get_manual_Trotter(num_q, spin_chain_hamiltonian, 0.1, n_trotter=2, trotter_type='symmetric').draw())


<a id='Script4pt33'></a>
## Script 4.33:

In [ ]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import transpile

# specifying a quantum register with specific number of qubits
qr = QuantumRegister(num_q)
# classical register used for measurement of qubits
cr = ClassicalRegister(num_q)
# quantum circuit combining quantum and classical registers
qc = QuantumCircuit(qr, cr) # instantiated here
qc.draw(style='iqp')
print(qc)


<a id='Script4pt34'></a>
## Script 4.34:

In [ ]:
# specifying initial state by flipping qubit states
for qubit_idx in range(num_q):
    if qubit_idx == 0:
        # generate only one spin-up at first qubit
        qc.id(qubit_idx)
    else:
        # make all other spins have the spin-down state
        qc.x(qubit_idx)
qc.barrier()
qc.draw(style='iqp')
print(qc)

# checking the initial state
device = Aer.get_backend('statevector_simulator')
qc_init_state = execute(qc, backend=device).result()
qc_init_state = qc_init_state.get_statevector()
print(qc_init_state)


<a id='Script4pt35'></a>
## Script 4.35:

In [ ]:
qr_init = QuantumRegister(num_q)
qc_init = QuantumCircuit(qr_init)
qc_init.initialize('011')
qc.append(qc_init, qc.qubits)


<a id='Script4pt36'></a>
## Script 4.36:

In [ ]:
# generating the time evolution operator for a specific set of
# hamiltonian parameters and timestep
time_evo_op = get_time_evolution_operator(num_qubits=num_q,
        tau=evolution_timestep,
        trotter_steps=n_trotter_steps,
        coeff=ham_coeffs)

# appending the Hamiltonian evolution to the circuit
qc.append(time_evo_op, list(range(num_q)))
qc.barrier()
qc.draw(style='iqp')
print(qc)

# Depth check
print('Depth of the circuit is', qc.depth())
# transpiled circuit to statevector simulator
qct = transpile(qc, device, optimization_level=2)
qct.decompose().decompose()
qct.draw(style='iqp')
print(qct)

print('Depth of the circuit after transpilation is '
        f'{qct.depth()}')


<a id='Script4pt37'></a>
## Script 4.37:

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import Aer

def qsolve_statevector(psin, qc):
    r'''
        Performs iterative quantum state propagation using a statevector simulator. The initial state is the statevector from the prior iteration:

        | \psi _t \rangle  = e^{i*\tau*H/hbar} e^{i*\tau*H/hbar} ... | \psi _0 \rangle
        -> | \psi _t \rangle  = e^{i*\tau*H/hbar} | \psi _{t-\tau} \rangle


        Args:
            psin (array): Initial quantum state.
            qc (QuantumCircuit): Circuit representing the time evolution operator.

        Returns:
            psin (statevector): final statevector after execution
    '''
    # Determining number of qubits from the length of the state vector
    n=np.size(psin)
    num_qubits=int(np.log2(np.size(psin)))
    # Circuit preparation
    qreg = QuantumRegister(num_qubits)
    circ = QuantumCircuit(qreg)

    circ.initialize(psin,qreg)
    circ.barrier()
    circ.append(qc, qreg)
    circ.barrier()

    # Circuit execution
    device = Aer.get_backend('statevector_simulator')
    psin = execute(circ, backend=device).result()
    return psin.get_statevector()


<a id='Script4pt38'></a>
## Script 4.38:

In [ ]:
# Qubit basis states
zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])

# Prepare an initial state (e.g., |011>), as follows
psin = zero_state # for the first spin
# iterates over the remaining spins, by performing
# Kronecker Product
for i in range(num_q-1):
    psin = np.kron(psin, one_state)
psin0 = psin.flatten()
print(psin0)

# time evolution operator
time_evo_op = get_time_evolution_operator(num_qubits=num_q,
        tau=evolution_timestep,
        trotter_steps=n_trotter_steps,
        coeff=ham_coeffs)
# number of steps for which to propagate
# (totaling 25 units of time)
nsteps = 250
psin_list = []
psin_list.append(psin0)
correlation_list = []

# Perform propagation by statevector re-initialization
for k in trange(nsteps):
    #print(f'Running dynamics step {k}')
    if k > 0:
        psin = qsolve_statevector(psin_list[-1], time_evo_op)
        # removes the last initial state to save memory
        psin_list.pop()
        # stores the new initial state
        psin_list.append(psin)
    correlation_list.append(np.vdot(psin_list[-1],psin0))

time = np.arange(0, evolution_timestep*(nsteps),
                 evolution_timestep)
np.save(f'{num_q}_spin_chain_time', time)
sa_observable = np.abs(correlation_list)
np.save(f'{num_q}_spin_chain_SA_obs', sa_observable)

# Plot survival amplitude
plt.plot(time, sa_observable, '-o')
plt.xlabel('Time')
plt.ylabel('Absolute Value of Survival Amplitude, '
           r'$\left|\langle \psi | \psi \rangle \right|$')
plt.xlim((min(time), max(time)))
plt.yscale('log')
plt.legend()
plt.show()


<a id='Script4pt39'></a>
## Script 4.39:

In [ ]:

def nested_kronecker_product(pauli_str):
    '''
    Computes the Kronecker Product for a given Pauli string (i.e., the pauli_str = 'ZZX' becomes the kronecker product Z Z X).

    Args:
        pauli_str (str): A string representation of Pauli matrices (e.g., 'ZZX').

    Returns:
        np.array: Resulting matrix after applying the Kronecker product.
    '''
    import numpy as np
    X = np.array([[0,1],[1,0]])
    Y = np.array([[0,complex(0,-1)],[complex(0,1),0]])
    Z = np.array([[1,0],[0,-1]])
    I = Z@Z

    # Define a dictionary with the four Pauli matrices:
    pms = {'I': I,'X': X,'Y': Y,'Z': Z}

    result = np.eye(1)  # Start with identity (size 1)
    for char in range(len(pauli_str)):
     result = np.kron(result,pms[pauli_str[char]])

    return result


<a id='Script4pt40'></a>
## Script 4.40:

In [ ]:
def decompose(Ham_arr, tol=1E-5):
    '''
    Decomposes a Hamiltonian matrix into a sum of Pauli strings.

    Args:
        Ham_arr (np.array): The input Hamiltonian matrix.
        tol (float): Tolerance for small coefficients (default: 1E-5).

    Returns:
        dict: Dictionary mapping Pauli strings to their coefficients.
    '''
    import numpy as np
    import itertools

    pauli_keys = ['I','X','Y','Z'] # Keys of the dictionary

    nqb = int(np.log2(Ham_arr.shape[0])) # Determine the numnber of qubits

    # Generate all possible Pauli strings
    sigma_combinations = list(itertools.product(pauli_keys, repeat=nqb))

    result = {} # Initialize an empty dictionary to the results
    for ii in range(len(sigma_combinations)):
        pauli_str = ''.join(sigma_combinations[ii])

        # Compute the Kronecker product of the corresponding Pauli matrices
        tmp_p_matrix = nested_kronecker_product(pauli_str)

        # Compute the coefficient using the Hilbert-Schmidt inner product
        a_coeff = (1/(2**nqb)) * np.trace(tmp_p_matrix @ Ham_arr)

        # Store only non-negligible coefficients
        if abs(a_coeff) > tol:
            result[pauli_str] = a_coeff.real

    return result


<a id='Script4pt41'></a>
## Script 4.41:

In [ ]:
from qflux.closed_systems.spin_dynamics_oo import SpinDynamicsH

num_q = 3
evolution_timestep = 0.1
n_trotter_steps = 1
hamiltonian_coefficients = [[0.75 / 2, 0.75 / 2, 0.0, 0.65]] + [
                            [0.5, 0.5, 0.0, 1.0] for _ in range(num_q - 1)
                            ]
initial_state = "011" # Specify the initial state as a binary string

qsimulation = SpinDynamicsH(
                    num_q,
                    evolution_timestep,
                    n_trotter_steps,
                    hamiltonian_coefficients,
                    )
qsimulation.run_simulation(state_string=initial_state, total_time=25, num_shots=100)
qsimulation.save_results('hadamard_test')
qsimulation.plot_results('hadamard_test')

Finished step 95: Re = -0.186, Im = -0.098
Running dynamics step 96


<a id='Script4pt42'></a>
## Script 4.42:

In [ ]:
import numpy as np
from qiskit_aer import Aer
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, ClassicalRegister

def get_hadamard_test(num_q, initial_state, control_operation,
                      control_repeats=0, imag_expectation=False):

    # Create circuit with quantum and classical registers
    qr_hadamard = QuantumRegister(num_q+1)
    cr_hadamard = ClassicalRegister(1)
    qc_hadamard = QuantumCircuit(qr_hadamard, cr_hadamard) # instantiated here

    # Initialize the computation qubits
    qc_hadamard.append(initial_state, qr_hadamard[1:]) # initial psi
    qc_hadamard.barrier()

    # Hadamard test on the ancilla qubit
    qc_hadamard.h(0)
    if imag_expectation:
        qc_hadamard.p(-np.pi/2, 0) # qc_hadamard.s(0).inverse() may be equivalent

    # iterates over the number of times the control operation should be added
    for i in range(control_repeats):
        qc_hadamard.append(control_operation, qr_hadamard[:])
    qc_hadamard.h(0)
    qc_hadamard.barrier()

    # Measuring the ancilla
    qc_hadamard.measure(0,0)

    return qc_hadamard


<a id='Script4pt43'></a>
## Script 4.43:

In [ ]:
def get_spin_correlation(counts):
    qubit_to_spin_map = {
        '0': 1,
        '1': -1,
    }
    total_counts = 0
    values_list = []
    for k,v in counts.items():
        values_list.append(qubit_to_spin_map[k] * v)
        total_counts += v
    # print(values_list)
    average_spin = (sum(values_list)) / total_counts
    return average_spin


<a id='Script4pt44'></a>
## Script 4.44:

In [ ]:
# IMPORTANT: Use qasm_simulator to obtain meaningful statistics
simulator = Aer.get_backend('qasm_simulator')

num_q = 3
n_trotter_steps = 1
# XX YY ZZ, Z
hamiltonian_coefficients = ([[0.75/2, 0.75/2, 0.0, 0.65]]
                            + [[0.5, 0.5, 0.0, 1.0]
                                for i in range(num_q-1)])

num_shots = 100 # increase to check for convergence

evolution_timestep = 0.1
total_time = 25
time_range = np.arange(0, total_time+evolution_timestep,
                       evolution_timestep)

# time evolution operator
time_evo_op = get_time_evolution_operator(num_qubits=num_q,
        tau=evolution_timestep,
        trotter_steps=n_trotter_steps,
        coeff=hamiltonian_coefficients)

controlled_time_evo_op = time_evo_op.control()
print(controlled_time_evo_op.decompose())

init_state_list = '1' + '0' * (num_q-1)
init_circ = get_initialization(num_q, init_state_list)
init_circ.draw(style='iqp')
print(init_circ)


<a id='Script4pt45'></a>
## Script 4.45:

In [ ]:
# it takes >1hr for 3 spins, with the parameters defined above
# lists t store observables
real_amp_list = []
imag_amp_list = []
for idx,time in enumerate(time_range):
    print(f'Running dynamics step {idx}')
    # Real component ------------------------------
    qc_had_real = get_hadamard_test(num_q, init_circ,
                                    controlled_time_evo_op,
                                    control_repeats=idx,
                                    imag_expectation=False)
    had_real_counts = get_circuit_execution_counts(
            qc_had_real, simulator, n_shots=num_shots)
    real_amplitude = get_spin_correlation(had_real_counts)
    real_amp_list.append(real_amplitude)

    # Imag component ------------------------------
    qc_had_imag = get_hadamard_test(num_q, init_circ,
                                    controlled_time_evo_op,
                                    control_repeats=idx,
                                    imag_expectation=True)
    had_imag_counts = get_circuit_execution_counts(
            qc_had_imag, simulator, n_shots=num_shots)
    imag_amplitude = get_spin_correlation(had_imag_counts)
    imag_amp_list.append(imag_amplitude)
    print(f'Finished step {idx}, where '
            f'Re = {real_amplitude:.3f} '
            f'Im = {imag_amplitude:.3f}')

    real_amp_array = np.array(real_amp_list)
    imag_amp_array = np.array(imag_amp_list)

np_abs_correlation_with_hadamard_test = np.abs(real_amp_array + 1j*imag_amp_array)

# plotting the data
plt.plot(time_range, np_abs_correlation_with_hadamard_test,
            '.', label='Hadamard Test')

sa_statevector = np.load(f'data/Part_I_SpinChain/{num_q}_spin_chain_SA_obs.npy')
time = np.load(f'{num_q}_spin_chain_time.npy')
plt.plot(time, sa_statevector, '-', label='Statevector')

plt.xlabel('Time')
plt.ylabel('Absolute Value of Survival Amplitude')
plt.legend()
plt.show()



<a id='Script5pt1'></a>
## Script 5.1:

In [ ]:
# -- Imports --
# - EfficientSU2: A parameterized quantum circuit (ansatz) often used in VQE.
# - SparsePauliOp: Efficient representation of Hamiltonians in terms of Pauli strings.
# - StatevectorEstimator: Estimates expectation values (ideal, noiseless backend).
import numpy as np
from scipy.optimize import minimize
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# --- Define the Hamiltonian ---
# H = 0.5 * Z_0 + 0.5 * Z_1 + 0.2 * X_0 * X_1
hamiltonian = SparsePauliOp.from_list([("ZI", 0.5), ("IZ", 0.5), ("XX", 0.2)])

# --- Initialize the Estimator Primitive ---
# StatevectorEstimator: Ideal, noiseless estimator using statevectors (no sampling noise). Computes expectation value exactly.
estimator = StatevectorEstimator()

# --- Define the Ansatz ---
# Use EfficientSU2 as a general-purpose parameterized ansatz. EfficientSU2 is expressive and hardware-efficient, using layers of single-qubit rotations and entangling gates.
ansatz = EfficientSU2(num_qubits=hamiltonian.num_qubits)

# --- Define the Energy Evaluation Function ---
# Given parameters params, assigns them to the ansatz circuit and evaluates the expectation value of the Hamiltonian. Returns the energy to the optimizer. Includes basic exception handling for robustness.
def energy(params, ansatz, hamiltonian, estimator):
    """Evaluate energy for given ansatz parameters."""
    try:
        result = estimator.run([(ansatz, hamiltonian, params)]).result()
        energy_estimate = result[0].data.evs
        print(f"Energy: {energy_estimate}")
        return energy_estimate
    except Exception as e:
        print(f"Estimator failed: {e}")
        return np.inf

# --- Initialize Parameters ---
# Random initialization of ansatz parameters in the full 0–2\pi range. Good starting point for global exploration of energy landscape.
initial_params = np.random.uniform(0, 2 * np.pi, size=ansatz.num_parameters)

# --- Classical Optimization ---
# Minimize energy using COBYLA, a derivative-free classical optimizer. Minimizes the energy function over the variational parameters. Hybrid quantum-classical loop: quantum subroutine evaluates energy, classical subroutine updates parameters.
opt_result = minimize(
    energy,
    initial_params,
    args=(ansatz, hamiltonian, estimator),
    method="COBYLA",
    options={"maxiter": 200, "disp": True}
)

# --- Output Results ---
# Outputs the final optimized parameters and estimated ground state energy.
final_params = opt_result.x
final_energy = energy(final_params, ansatz, hamiltonian, estimator)

print(f"\nFinal Optimized Energy: {final_energy}")
print(f"Optimized Parameters: {final_params}")

<a id='Script6pt1'></a>
## Script 6.1:

In [ ]:
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qflux.closed_systems.VarQTE import VarQRTE, Construct_Ansatz

# --- Define the Hamiltonian ---
H = SparsePauliOp.from_list([("X", 1.0)])

# --- Define the Initial State ---
qc = QuantumCircuit(1)
qc.x(0) # Creates |1> state

# --- Perform Variational Real-Time Evolution ---
layers = 1
total_time = 12
timestep = 0.1
params = VarQRTE(layers, H, total_time, timestep, init_circ=qc)
# Params now holds the parameter values for the ansatz at each timestep

# --- Get the Expectation Values of an Observable ---
estimator = Estimator()
observable = SparsePauliOp.from_list([("Z", 1.0)])
spin1_values = []
spin2_values = []
for i in range(len(params)):
    ansatz = Construct_Ansatz(qc, params[i], H.num_qubits)
    result = estimator.run(ansatz, observables=observable).result()
    spin1_values.append(result.values[0])

# --- Plot Expectation Values Over Time ---
plt.title("Spin Expectation Value Over Time")
plt.plot([i*timestep for i in range(int(total_time/timestep)+1)], spin1_values)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.show()

<a id='Script6pt2'></a>
## Script 6.2:

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp


# To change the ansatz, apply_param and measure_der must both be modified.
def apply_param(
    params: npt.NDArray[np.float64], i: int, qc: QuantumCircuit, N: int
) -> None:
    """Apply parameter i to the quantum circuit currently constructing the ansatz.
        The ansatz must be built in a peicewise manner to allow for hadamard tests
        of the generators of the parameters to later be inserted to measure the A_ij
        and C_i matrix elements.

    Args:
        params (numpy.array): An array containing the values of all the ansatz parameters.
        parameter (int): Index of the parameter being applied.
        qc (QuantumCircuit): The qiskit ansatz quantum circuit currently being constructed.
        N (int): Number of qubits
    """
    qc.rx(params[i], i % N)
    if i % N == N - 1 and i != len(params) - 1:
        for i in range(N - 1):
            qc.cz(i, i + 1)


def measure_der(i: int, qc: QuantumCircuit, N: int) -> None:
    """Append a Hadamard test to the circuit to measure the generator of parameter i in the ansatz.
        The ansatz currently used is simply the two-local ansatz with only rx gates.
        Therefore the generator is only x-gates on the corresponding qubit.

    Args:
        parameter (int): The index of the parameter whose generator will be measured.
        qc (QuantumCircuit): The qiskit quantum circuit which is in the process of assembling the ansatz.
        N (int): Number of qubits
    """
    qc.cx(N, i % N)


def pauli_measure(qc: QuantumCircuit, pauli_string: str) -> None:
    """Measure the given pauli string on the provided quantum circuit using a hadamard test.

    Args:
        qc (QuantumCircuit): The quantum circuit ansatz being constructed.
        pauli_string (str): The pauli string to be measured as a string.
    """
    N = len(pauli_string)
    for i in range(len(pauli_string)):  # Measure Pauli Strings
        if str(pauli_string[i]) == "X":
            qc.cx(N, i)
        if str(pauli_string[i]) == "Y":
            qc.cy(N, i)
        if str(pauli_string[i]) == "Z":
            qc.cz(N, i)

<a id='Script6pt3'></a>
## Script 6.3:

In [ ]:
def A_Circuit(params: npt.NDArray[np.float64], i: int, j: int, N: int) -> QuantumCircuit:
    """Constructs the qiskit quantum circuits used to measure each element of the A_ij matrix.

    Args:
        params (numpy.array): A numpy array containing the values of each parameter of the ansatz.
        i (int): The index from A_ij.  This also corresponds to the ansatz parameter i being measured.
        j (int): The index from A_ij.  This also corresponds to the ansatz parameter j being measured.
        N (int): The number of qubits.

    Returns:
        QuantumCircuit: The quantum circuit for an element of the A_ij matrix, in the form of a
            hadamard test of the generators of parameters i and j.  The value of the A_ij matrix
            can be found by measuring the ancilla qubit (qubit N) in the Z basis.
    """
    qc = QuantumCircuit(N + 1, 1)
    qc.h(N)
    for parameter in range(len(params)):  # Apply parameterized gates
        if parameter == i:
            qc.x(N)
            measure_der(parameter, qc, N)  # Measure generator for i
            qc.x(N)
        if parameter == j:
            measure_der(parameter, qc, N)  # Measure second generator for j
        apply_param(params, parameter, qc, N)
    qc.h(N)
    return qc


def Measure_A(
    init_circ: QuantumCircuit,
    params: npt.NDArray[np.float64],
    N: int,
    shots: int = 2**10,
    noisy: bool = False,
) -> npt.NDArray[np.float64]:
    """Create the A_ij matrix through measuring quantum circuits corresponding to each element.

    Args:
        init_circ (QuantumCircuit): The qiskit circuit representing the initial state of the system.
        params (numpy.array): A numpy array which contains the values of each parameter of the ansatz.
        N (int): The number of qubits.
        shots (int, optional): The number of shots used to estimate each element of the A_ij matrix. Defaults to 2**10.
        noisy (bool, optional): A boolean used to turn on and off the Fake-Sherbrooke qiskit noisy backend. Defaults to False.

    Returns:
        numpy.array: The A_ij matrix
    """
    A = [[0.0 for i in range(len(params))] for j in range(len(params))]
    for i in range(len(params)):
        for j in range(len(params) - i):
            qc = QuantumCircuit(N + 1, 1)
            ansatz = A_Circuit(params, i, i + j, N)
            qc = qc.compose(init_circ, [k for k in range(N)])
            qc = qc.compose(ansatz, [k for k in range(N + 1)])

            observable = SparsePauliOp.from_list([("Z" + "I" * N, 1.0)])
            if noisy:
                device_backend = FakeSherbrooke()
                coupling_map = device_backend.coupling_map
                noise_model = NoiseModel.from_backend(device_backend)
                basis_gates = noise_model.basis_gates
                estimator = Estimator(options={
                                    "backend_options":{"noise_model": noise_model},
                                    "run_options":{"shots": shots}}
                )
            else:
                estimator = Estimator(options={"run_options":{"shots": shots}})
            result = estimator.run([(qc, observable)]).result()
            A[i][i + j] = result[0].data.evs
    return np.array(A)

<a id='Script6pt4'></a>
## Script 6.4:

In [ ]:
def C_Circuit(
    params: npt.NDArray[np.float64],
    i: int,
    pauli_string: str,
    N: int,
    evolution_type: str = "real",
) -> QuantumCircuit:

    """Create the qiskit quantum circuits to measure each element of the C_i vector.

    Args:
        params (numpy.array): A numpy array which contains the values of each parameter of the ansatz.
        i (int): The index of the C_i vector being measured. This also corresponds
            to the index i of the parameter whose generator will be measured
        pauli_string (str): A string containing a description of the pauli operator of the Hamiltonian which will be measured.
        N (int): The number of qubits.
        evolution_type (str, optional): This determines if the evolution will be real-time or imaginary-time
            through the addition of an extra gate. Defaults to "real".

    Returns:
        QuantumCircuit: The quantum circuit for an element of the C_i matrix, in the form of a
            hadamard test of the generators of parameter i.  The value of the C_i matrix
            can be found by measuring the ancilla qubit (qubit N) in the Z basis.
    """
    qc = QuantumCircuit(N + 1, 1)
    qc.h(N)
    if evolution_type == "imaginary":
        qc.s(N)  # To get only imaginary component
    else:
        qc.z(N)
    for parameter in range(len(params)):  # Apply parameterized gates
        if parameter == i:
            qc.x(N)
            measure_der(parameter, qc, N)  # Measure generators
            qc.x(N)
        apply_param(params, parameter, qc, N)
    pauli_measure(qc, pauli_string)
    qc.h(N)
    return qc


def Measure_C(
    init_circ: QuantumCircuit,
    params: npt.NDArray[np.float64],
    H: SparsePauliOp,
    N: int,
    shots: int = 2**10,
    evolution_type: str = "real",
    noisy: bool = False,
) -> npt.NDArray[np.float64]:
    """Create the C_i vector through measuring quantum circuits corresponding to each element.

    Args:
        init_circ (QuantumCircuit): A qiskit circuit constructing the initial state of the system.
        params (numpy.array): A numpy array containing the values of the parameters of the ansatz.
        H (SparsePauliOp): The Hamiltonian.
        N (int): The number of qubits.
        shots (int, optional): The number of shots to be used to measure each element of the C_i vector. Defaults to 2**10.
        evolution_type (str, optional): This determines if the evolution will be real-time or imaginary-time
            through the addition of an extra gate. Defaults to "real".
        noisy (bool, optional): A boolean used to turn on and off the Fake-Sherbrooke qiskit noisy backend. Defaults to False.

    Returns:
        numpy.array: The C_i vector.
    """
    C = [0.0 for i in range(len(params))]
    for i in range(len(params)):
        for pauli_string in range(len(H.paulis)):
            qc = QuantumCircuit(N + 1, 1)
            ansatz = C_Circuit(
                params, i, H.paulis[pauli_string], N, evolution_type=evolution_type
            )
            qc = qc.compose(init_circ, [k for k in range(N)])
            qc = qc.compose(ansatz, [k for k in range(N + 1)])
            observable = SparsePauliOp.from_list([("Z" + "I" * N, 1.0)])
            if noisy:
                device_backend = FakeSherbrooke()
                coupling_map = device_backend.coupling_map
                noise_model = NoiseModel.from_backend(device_backend)
                basis_gates = noise_model.basis_gates
                estimator = Estimator(options={
                                        "backend_options":{"noise_model": noise_model},
                                        "run_options":{"shots": shots}}
                    )
            else:
                estimator = Estimator(options={"run_options":{"shots": shots}})
            result = estimator.run([(qc, observable)]).result()

            C[i] -= 1 / 2 * H.coeffs[pauli_string].real * result[0].data.evs
    return np.array(C)

<a id='Script6pt5'></a>
## Script 6.5:

In [ ]:
def VarQRTE(
    n_reps_ansatz: int,
    hamiltonian: SparsePauliOp,
    total_time: float = 1.0,
    timestep: float = 0.1,
    init_circ: Optional[QuantumCircuit] = None,
    shots: int = 2**10,
    noisy: bool = False,
) -> List[npt.NDArray[np.float64]]:
    """The Variational Quantum Real Time Evolution (VarQRTE) algorithm.  This uses quantum circuits to measure
        the elements of two objects, the A_ij matrix and the C_i vector.

    Args:
        n_reps_ansatz (int): The number of repetitions of the variational ansatz used to simulate Real-Time evolution.
        hamiltonian (SparsePauliOp): The Hamiltonian of the system.
        total_time (float, optional): A float to determine the total evolution time of the quantum system. Defaults to 1.0.
        timestep (float, optional): A float to determine the size of a single timestep. Defaults to 0.1.
        init_circ (QuantumCircuit, optional): A qiskit circuit constructing the initial state of the system.. Defaults to None.
        shots (int, optional): Number of shots to be used to measure observables. Defaults to 2**10.
        noisy (bool, optional): A boolean used to turn on and off the Fake-Sherbrooke qiskit noisy backend. Defaults to False.

    Returns:
        numpy.array: An array containing all the parameter values of the ansatz throughout its time evolution.
            These values can be put into Construct_Ansatz, or anstaz_energy to obtain observables of the system.
    """
    if init_circ is None:
        init_circ = QuantumCircuit(hamiltonian.num_qubits)

    initial_params = np.zeros(hamiltonian.num_qubits * (n_reps_ansatz + 1))
    num_timesteps = int(total_time / timestep)
    all_params = [np.copy(initial_params)]
    my_params = np.copy(initial_params)  # Reset Initial Parameters after each run
    for i in range(num_timesteps):
        print(f"Simulating Time={str(timestep*(i+1))}                      ", end="\r")
        theta_dot = np.array([0.0 for j in range(len(my_params))])
        A = Measure_A(
            init_circ, my_params, hamiltonian.num_qubits, shots=shots, noisy=noisy
        )
        C = Measure_C(
            init_circ,
            my_params,
            hamiltonian,
            hamiltonian.num_qubits,
            shots=shots,
            evolution_type="real",
            noisy=noisy,
        )

        # Approximately invert A using Truncated SVD
        u, s, v = np.linalg.svd(A)
        for j in range(len(s)):
            if s[j] < 1e-2:
                s[j] = 1e8
        t = np.diag(s**-1)
        A_inv = np.dot(v.transpose(), np.dot(t, u.transpose()))

        theta_dot = np.matmul(A_inv, C)

        my_params -= theta_dot * timestep
        all_params.append(np.copy(my_params))
    return all_params


def Construct_Ansatz(
    init_circ: QuantumCircuit, params: npt.NDArray[np.float64], N: int
) -> QuantumCircuit:
    """Construct the full ansatz for use in measuring observables.

    Args:
        init_circ (QuantumCircuit): A qiskit circuit constructing the initial state of the system.
        params (numpy.array): A numpy vector containing the values of the parameters of the ansatz at a specific time.
        N (int): The number of qubits.

    Returns:
        QuantumCircuit: The full ansatz as a qiskit.QuantumCircuit.
    """
    qc = QuantumCircuit(N, 0)
    qc = qc.compose(init_circ, [k for k in range(N)])

    ansatz = QuantumCircuit(N, 0)
    for parameter in range(len(params)):  # Apply parameterized gates
        apply_param(params, parameter, ansatz, N)

    qc = qc.compose(ansatz, [k for k in range(N)])
    return qc

<a id='Script6pt6'></a>
## Script 6.6:

In [ ]:
def VarQITE(
    n_reps_ansatz: int,
    hamiltonian: SparsePauliOp,
    total_time: float,
    timestep: float,
    init_circ: Optional[QuantumCircuit] = None,
    shots: int = 2**10,
    noisy: bool = False,
) -> List[npt.NDArray[np.float64]]:
    """The Variational Quantum Imaginary Time Evolution (VarQITE) algorithm.  This uses quantum circuits to measure
        the elements of two objects, the A_ij matrix and the C_i vector.

    Args:
        n_reps_ansatz (int): The number of repetitions of the variational ansatz used to simulate Imaginary-Time evolution.
        hamiltonian (SparsePauliOp): The Hamiltonian of the system.
        total_time (float, optional): A float to determine the total evolution time of the quantum system. Defaults to 1.0.
        timestep (float, optional): A float to determine the size of a single timestep. Defaults to 0.1.
        init_circ (QuantumCircuit, optional): A qiskit circuit constructing the initial state of the system.. Defaults to None.
        shots (int, optional): Number of shots to be used to measure observables. Defaults to 2**10.
        noisy (bool, optional): A boolean used to turn on and off the Fake-Sherbrooke qiskit noisy backend. Defaults to False.

    Returns:
        numpy.array: An array containing all the parameter values of the ansatz throughout its time evolution.
            These values can be put into Construct_Ansatz, or anstaz_energy to obtain observables of the system.
    """
    if init_circ is None:
        init_circ = QuantumCircuit(hamiltonian.num_qubits)

    initial_params = np.zeros(hamiltonian.num_qubits * (n_reps_ansatz + 1))
    num_timesteps = int(total_time / timestep)
    all_params = [np.copy(initial_params)]

    my_params = np.copy(initial_params)  # Reset Initial Parameters after each run
    for i in range(num_timesteps):
        print(f"Timestep: {str(i*timestep)}                      ", end="\r")
        theta_dot = np.array([0.0 for j in range(len(my_params))])
        A = np.array(
            Measure_A(
                init_circ, my_params, hamiltonian.num_qubits, shots=shots, noisy=noisy
            )
        )
        C = np.array(
            Measure_C(
                init_circ,
                my_params,
                hamiltonian,
                hamiltonian.num_qubits,
                shots=shots,
                noisy=noisy,
                evolution_type="imaginary",
            )
        )

        # Approximately invert A using Truncated SVD
        u, s, v = np.linalg.svd(A)
        for j in range(len(s)):
            if s[j] < 1e-2:
                s[j] = 1e7
        t = np.diag(s**-1)
        A_inv = np.dot(v.transpose(), np.dot(t, u.transpose()))
        # A_inv=np.dot(v,np.dot(t,u.transpose()))

        theta_dot = np.matmul(A_inv, C)

        my_params += theta_dot * timestep
        all_params.append(np.copy(my_params))

        # print("Theta dot: "+str(np.sum(np.abs(theta_dot))))
        # print("(Energy ,Variance): "+str(ansatz_energy(init_circ, my_params[:], hamiltonian)))
        # print()
    return all_params


def ansatz_energy(
    init_circ: QuantumCircuit,
    params: npt.NDArray[np.float64],
    H: SparsePauliOp,
    shots: int = 2**14,
    noisy: bool = False,
) -> Tuple[float, float]:
    """Measure the energy of the ansatz.

    Args:
        init_circ (QuantumCircuit): A qiskit circuit constructing the initial state of the system.
        params (numpy.array): A numpy vector containing the values of the parameters of the ansatz at a specific time.
        H (SparsePauliOp): The Hamiltonian.
        shots (_type_, optional): The number of shots to be used to measure the energy. Defaults to 2**14.
        noisy (bool, optional): A boolean used to turn on and off the Fake-Sherbrooke qiskit noisy backend. Defaults to False.

    Returns:
        (float, float): Return (energy, variance) from the measured observables.
    """
    N = H.num_qubits

    if noisy:
        device_backend = FakeSherbrooke()
        coupling_map = device_backend.coupling_map
        noise_model = NoiseModel.from_backend(device_backend)
        basis_gates = noise_model.basis_gates
        estimator = Estimator(options={
                                "backend_options":{"noise_model": noise_model},
                                "run_options":{"shots": shots}}
                                )
    else:
        estimator = Estimator(options={"run_options":{"shots": shots}})
    qc = Construct_Ansatz(init_circ, params, N)
    result = estimator.run([(qc, H)]).result()
    return result[0].data.evs, result[0].data.stds

<a id='Script6pt7'></a>
## Script 6.7:

In [ ]:
# --- Define the Hamiltonian ---
H = SparsePauliOp.from_list([("IIZ", 1.0), ("IZI", 1.0), ("ZII", 0.65), ("IXX", 1.0), ("IYY", 1.0), ("XXI", 0.75), ("YYI", 0.75)])

# --- Define the Initial State ---
qc = QuantumCircuit(3)
qc.rx(0.5, 0)
qc.rx(0.5, 1)
qc.rx(0.5, 2)

# --- Perform Variational Real-Time Evolution ---
layers = 0
total_time = 10
timestep = 0.1
params = VarQITE(layers, H, total_time, timestep, init_circ=qc)
# Params now holds the parameter values for the ansatz at each timestep for Imaginary-Time Evolution

# --- Get the Expectation Value of the Energy ---
all_energies = []
for i in range(len(params)):
    print(f"Timestep {i} Energy: {ansatz_energy(qc, params[i], H)}")
    all_energies.append(ansatz_energy(qc, params[i], H)[0])

# --- Plot Expectation Values Over Time ---
plt.title("VarQITE Energy Over Imaginary Time")
plt.plot([i*timestep for i in range(int(total_time/timestep)+1)], all_energies)
plt.xlabel("Imaginary Time")
plt.ylabel("Energy (eV)")
plt.show()